In [9]:
#! Install library
import os
import importlib.util

if importlib.util.find_spec("requests") is None:
	os.system("pip install requests")

if importlib.util.find_spec("beautifulsoup4") is None:
	os.system("pip install beautifulsoup4")

if importlib.util.find_spec("Counter") is None:
	os.system("pip install Counter")

In [10]:
#! Authentication details
import os
import dotenv
import importlib.util

JiraHost = None
ConfluenceHost = None
Username = None
Password = None

if importlib.util.find_spec("google.colab") is not None: ## if using google colab
    if not os.path.exists('.env'):
        from google.colab import files
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        try:
            os.rename(file_name, '.env')
        except:
            pass

try:
    dotenv.load_dotenv('../.env', override=True)

    JiraHost = os.getenv('SECRETS_HOST')
    ConfluenceHost = os.getenv('SECRETS_CONFLUENCE')
    Username = os.getenv('SECRETS_USERNAME')
    Password = os.getenv('SECRETS_PASSWORD')
except:
    display("trouble loading dot env")
    pass

if JiraHost is None or JiraHost == "":
    JiraHost = input("Enter Jira Host")

if ConfluenceHost is None or ConfluenceHost == "":
    ConfluenceHost = input("Enter Confluence Host")

if Username is None or Username == "":
    Username = input("Enter Username")

if Password is None or Password == "":
    Password = input("Enter Password")

display("Jira Host: " + JiraHost)
display("Confluence Host: " + ConfluenceHost)

'Jira Host: https://jira.budgetdirect.com.au/'

'Confluence Host: https://confluence.budgetdirect.com.au/'

In [11]:
#! Functions
import base64
import re
from bs4 import BeautifulSoup
import requests
from collections import Counter
import pandas as pd
import requests
from functools import reduce
import warnings

requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)

def _ExpandColumn(self:pd.DataFrame, colName:str, columnsToExpand = [], prefix:str = "Prefix", sentenceCase:bool = True) -> pd.DataFrame:
    if (prefix == "Prefix"):
        prefix = colName + " "
        with warnings.catch_warnings():
          warnings.simplefilter(action='ignore', category=FutureWarning)
          expandedCols = self[colName].apply(lambda x: pd.Series(x).add_prefix(prefix))
        columnsToExpand = [prefix + c for c in columnsToExpand]
    else:
        expandedCols = self[colName].apply(lambda x: pd.Series(x))

    if len(columnsToExpand) > 0:
        expandedCols = expandedCols[columnsToExpand]

    if sentenceCase:
        expandedCols.columns = [fnSentenceCase(c) for c in expandedCols.columns]

    return pd.concat([self.drop(colName, axis=1), expandedCols], axis=1)

pd.DataFrame.expand = _ExpandColumn

def fnSentenceCase(s):
    s = (' '.join(dict.fromkeys(s.split())))  # remove duplicate words
    s = s.replace("0", "") # remove "0"
    s = s.strip()
    return ' '.join([x.capitalize() for x in re.sub(r"([A-Z])", r" \1", s).split()]) # sentence case

def _SentenceCaseColumns(self:pd.DataFrame) -> pd.DataFrame:
    self.columns = [fnSentenceCase(c) for c in self.columns]
    return self

pd.DataFrame.sentence_case_columns = _SentenceCaseColumns

def fnGetDefaultHeaders():
    return {
        "content-type": "application/json",
        "authorization": "Basic " + base64.b64encode((Username + ":" + Password).encode()).decode(),
        "retry-after": "120"
    }

def fnAPI(webRequestDelegate, startAt = 0) -> pd.DataFrame:
    def flatten_reduce_lambda(frm):
        try:
            return list(reduce(lambda x, y: x + y, frm, []))
        except:
            return list(reduce(lambda x, y: x + y, [frm], []))
    def innerGetResults(webRequestDelegate, startAt = 0):
        results = webRequestDelegate(startAt)
        if isinstance(results, dict) and "total" in results and "maxResults" in results:
            if startAt + results["maxResults"] < results["total"]:
                return [results] + innerGetResults(webRequestDelegate, startAt + results["maxResults"])
            else:
                return [results]
        else:
            return [results]
    Source = flatten_reduce_lambda(innerGetResults(webRequestDelegate, startAt))
    df = pd.DataFrame(Source)
    return df

def most_frequent(List):
	try:
		c = Counter(List)
		most_common = [key for key, _ in c.most_common(5)]
		return most_common
	except:
		return ""

In [12]:
#! Test Myself API Call
from jira import JIRA

print(JiraHost, Username, Password)

def fnMyself():
    def ApiCall():
        url = "/rest/api/2/myself"
        headers = fnGetDefaultHeaders()
        response = requests.get(
            JiraHost + url,
            headers=headers,
            verify=False,
            allow_redirects=True,
        )
        print(response)
        return response.json()

    df = ApiCall()
    return df


print(fnMyself())

https://jira.budgetdirect.com.au/ powerbi AutoGeneral2020!
<Response [200]>
{'self': 'https://jira.budgetdirect.com.au/rest/api/2/user?username=powerbi', 'key': 'powerbi', 'name': 'powerbi', 'emailAddress': 'powerbi@autogeneral.com.au', 'avatarUrls': {'48x48': 'https://jira.budgetdirect.com.au/secure/useravatar?ownerId=powerbi&avatarId=26725', '24x24': 'https://jira.budgetdirect.com.au/secure/useravatar?size=small&ownerId=powerbi&avatarId=26725', '16x16': 'https://jira.budgetdirect.com.au/secure/useravatar?size=xsmall&ownerId=powerbi&avatarId=26725', '32x32': 'https://jira.budgetdirect.com.au/secure/useravatar?size=medium&ownerId=powerbi&avatarId=26725'}, 'displayName': 'Power Bi', 'active': True, 'deleted': False, 'timeZone': 'Australia/Brisbane', 'locale': 'en_AU', 'groups': {'size': 21, 'items': []}, 'applicationRoles': {'size': 1, 'items': []}, 'expand': 'groups,applicationRoles'}


In [24]:
import pandas as pd
import json

def fnGetProjects():
    def ApiCall():
        url = "/rest/api/latest/project"
        headers = fnGetDefaultHeaders()
        response = requests.get(
            JiraHost + url,
            headers=headers,
            verify=False,
            allow_redirects=True,
        )
        return response.json()

    Source = ApiCall()
    df = pd.DataFrame(Source)
    return df

def fnSetProjectPermissionScheme(projectKey, schemeId):
    def ApiCall(p, s):
        url = "rest/api/latest/project/" + p + "/permissionscheme"
        headers = fnGetDefaultHeaders()
        response = requests.put(
            JiraHost + url,
            headers=headers,
            data='{"id":' + str(s) + "}",
            verify=False,
            allow_redirects=True,
        )
        return response.json()

    df = ApiCall(projectKey, schemeId)
    return df


def fnGetPermissionSchemes() -> pd.DataFrame:
    def ApiCall(startAt):
        url = "/rest/api/latest/permissionscheme"
        headers = fnGetDefaultHeaders()
        response = requests.get(
            JiraHost + url,
            headers=headers,
            verify=False,
            allow_redirects=True,
        )
        return response.json()

    df = fnAPI(ApiCall)
    df = df.explode("permissionSchemes")
    df = df.expand("permissionSchemes")
    return df

dfs = fnGetPermissionSchemes()
dfs = dfs[dfs["Permission Schemes Name"] == "READONLY"]

scheme = int(dfs.iloc[0]["Permission Schemes Id"])

df = fnGetProjects()
##df = df[df["Permission Schemes Id"] == 10001]
# display(df.head())

update = 0

for index, row in df.iterrows():
    try:
        result = fnSetProjectPermissionScheme(row["key"], scheme)
        update = update + 1
    except Exception as ex:
        display(["error", ex, row["key"]])

print("Updated = ", update)
print("Total = ", len(df))

,Permission Schemes Expand,Permission Schemes Id,Permission Schemes Self,Permission Schemes Name,Permission Schemes Description
0,"permissions,user,group,projectRole,field,all",14880,https://jira.budgetdirect.com.au/rest/api/2/pe...,READONLY,Used to lock out all users from creating new c...


'14880'

Updated =  563
Total =  563
